In [43]:
from datetime import datetime, timedelta
from tkinter import messagebox
import tkinter as tk
import pandas as pd
import numpy as np
import locale
import warnings
import os

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings('ignore')

hoy = datetime.now()
año = hoy.year
mes = hoy.month

mes_nombre = hoy.strftime('%b').upper()[:3] # ENE
mes_año = f'{mes_nombre}{str(año)[2:]}' # ENE24
año_mes = f'{año}{str(mes).zfill(2)}' # 202401

In [44]:
def get_file_name() -> list[str]:
    file_list = []
    for file in os.listdir(f'bases/pagos/{año_mes}'):
        if file.endswith('.xlsx') and 'Base Pagos' in file:
            file_list.append(file)
    return file_list

def get_last_date(dates_list: list[datetime]) -> datetime:
    last_date = dates_list[0]
    for date in dates_list:
        if date > last_date:
            last_date = date
    return last_date

def get_date(file_list: list[str]) -> str:
    dates_list = []
    for file in file_list:
        fecha_list =  file.split(' ')[-1].split('.')[:3]
        dates_list.append(datetime.strptime(f'{fecha_list[0]}.{fecha_list[1]}.{fecha_list[2]}', '%d.%m.%Y'))
    return get_last_date(dates_list).strftime('%d.%m.%Y')

fecha = get_date(get_file_name())
fecha

'04.11.2024'

In [45]:
def validar_o_crear_carpeta() -> str:
    fecha_hoy = datetime.today().strftime('%Y.%m.%d')
    
    carpeta_reporte = 'reporte'
    
    carpeta_hoy = os.path.join(carpeta_reporte, fecha_hoy)
    
    if not os.path.exists(carpeta_hoy):
        os.makedirs(carpeta_hoy)
        print(f'Carpeta creada: {carpeta_hoy}')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}')
    
    if not os.path.exists(f'{carpeta_hoy}/multi_agencias'):
        os.makedirs(f'{carpeta_hoy}/multi_agencias')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}\multi_agencias')
    
    return carpeta_hoy

file_path = validar_o_crear_carpeta()
file_path

La carpeta ya existe: reporte\2024.11.05
La carpeta ya existe: reporte\2024.11.05\multi_agencias


'reporte\\2024.11.05'

In [46]:
base_pagos_path = f'bases/pagos/{año_mes}/Base Pagos {fecha}.xlsx'
asignacion_path = f'bases/asignacion/{año_mes}/base_asignacion_{mes_año}.xlsx'

print(base_pagos_path)
print(asignacion_path)

monoproducto = f'{file_path}/MONOPRODUCTO {fecha}.xlsx'
multiproducto = f'{file_path}/MULTIPRODUCTO {fecha}.xlsx'
reactiva = f'{file_path}/REACTIVA {fecha}.xlsx'
no_enviados = f'{file_path}/NO ENVIADOS {fecha}.xlsx'
si_enviados = f'{file_path}/ENVIADOS {fecha}.xlsx'

mono_path = os.path.abspath(monoproducto)
multi_path = os.path.abspath(multiproducto)
react_path = os.path.abspath(reactiva)
no_enviados_path = os.path.abspath(no_enviados)
enviados_path = os.path.abspath(si_enviados)

print(mono_path)
print(multi_path)
print(react_path)
print(no_enviados_path)
print(enviados_path)

bases/pagos/202411/Base Pagos 04.11.2024.xlsx
bases/asignacion/202411/base_asignacion_NOV24.xlsx
c:\Users\p042833\Documents\GitHub\base-pagos\reporte\2024.11.05\MONOPRODUCTO 04.11.2024.xlsx
c:\Users\p042833\Documents\GitHub\base-pagos\reporte\2024.11.05\MULTIPRODUCTO 04.11.2024.xlsx
c:\Users\p042833\Documents\GitHub\base-pagos\reporte\2024.11.05\REACTIVA 04.11.2024.xlsx
c:\Users\p042833\Documents\GitHub\base-pagos\reporte\2024.11.05\NO ENVIADOS 04.11.2024.xlsx
c:\Users\p042833\Documents\GitHub\base-pagos\reporte\2024.11.05\ENVIADOS 04.11.2024.xlsx


In [47]:
#################### BASE DE PAGOS ####################

In [48]:
df_base = pd.read_excel(base_pagos_path)
print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(95, 9)
Entidad         int64
FECHA          object
CODCEN          int64
CLAVSERV        int64
CENTROPAGO      int64
IMPORTE       float64
MONEDA         object
NOMBRE         object
ESTADO         object
dtype: object


,Entidad,FECHA,CODCEN,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO
0,11,2024-11-02,21572302,248,145,1090.0,PEN,CRISTIAN MARCOS CHURA CAPCHA,P
1,11,2024-11-02,26476829,248,306,3000.0,PEN,001103065002979042 REYCOR EIRL,P
2,11,2024-11-02,25874888,248,109,2175.0,PEN,ALVAREZ VALENCIA VALERA ALEJANDRA,P
3,11,2024-11-04,24173081,248,301,2000.0,PEN,CAUDAL INVERSIONES EIRL .,P
4,11,2024-11-04,27358572,248,215,2000.0,PEN,00110215129600373540 DNI 42368321,P


In [49]:
fecha_formateada = pd.to_datetime('today').strftime('%d-%b')
fecha_formateada = fecha_formateada[:3] + fecha_formateada[3:].capitalize()

df_base['FECHA_ENVIO'] = fecha_formateada.replace('.', '')
df_base['ID_RESPONSABLE'] = 'MIV'

cols_base = ['FECHA', 'CODCEN', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_base = df_base[cols_base]
df_base.rename(columns={'CODCEN': 'CC'}, inplace=True)

df_base['CC'] = df_base['CC'].astype('Int64').astype(str).str.zfill(8)
df_base['CC'] = df_base['CC'].str.replace(' ', '').str.replace(r'\D', '', regex=True).str[-8:]
df_base['CC'] = df_base['CC'].str.zfill(8)

df_base['CLAVSERV'] = df_base['CLAVSERV'].astype(str).str.zfill(5)
df_base['CENTROPAGO'] = df_base['CENTROPAGO'].astype(str).str.zfill(5)

print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(95, 9)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO         object
IMPORTE           float64
MONEDA             object
NOMBRE             object
FECHA_ENVIO        object
ID_RESPONSABLE     object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,FECHA_ENVIO,ID_RESPONSABLE
0,2024-11-02,21572302,00248,00145,1090.0,PEN,CRISTIAN MARCOS CHURA CAPCHA,05-Nov,MIV
1,2024-11-02,26476829,00248,00306,3000.0,PEN,001103065002979042 REYCOR EIRL,05-Nov,MIV
2,2024-11-02,25874888,00248,00109,2175.0,PEN,ALVAREZ VALENCIA VALERA ALEJANDRA,05-Nov,MIV
3,2024-11-04,24173081,00248,00301,2000.0,PEN,CAUDAL INVERSIONES EIRL .,05-Nov,MIV
4,2024-11-04,27358572,00248,00215,2000.0,PEN,00110215129600373540 DNI 42368321,05-Nov,MIV


In [50]:
#################### ASIGNACION ####################

In [51]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar base de asignación?", icon='warning')
if result == 'yes':
    df_asignacion = pd.read_excel(asignacion_path)
    print(df_asignacion.shape)
    df_asignacion.head(5)

root.destroy()

(242479, 10)


In [52]:
#################### BACKUPS ####################

In [53]:
df_base_test = df_base.copy()
print('Base', df_base_test.shape)

df_asignacion_test = df_asignacion.copy()
print('Asignacion', df_asignacion_test.shape)

Base (95, 9)
Asignacion (242479, 10)


In [54]:
df_asignacion[df_asignacion['CONTRATO']==1101419600143586]

,CC,CONTRATO,NOMBRE_CLIENTE,MONEDA,CAPITAL,CARTERA,TIPO_FONDO,TIPO_CARTERA,AGENCIA,FLAG
176084,28366126,1101419600143586,FABMEDIC SAC,PEN,45847.18,KSTBC,REACTIVA,UNSECURED,ASESCOM RJ,3


In [55]:
base_count = df_base_test.shape[0]
base_count

95

In [56]:
def actualizar_tipo_cartera(grupo: pd.DataFrame) -> pd.DataFrame:
    grupo['TIPO_CARTERA'] = np.where(
        grupo['TIPO_CARTERA'].eq('UNSECURED').any(), 'UNSECURED',
        np.where(
            ~grupo['TIPO_CARTERA'].eq('NULL').any(), 'SECURED', 
            np.where(
                ~grupo['TIPO_CARTERA'].eq('SECURED').any(), 'NULL', 'SECURED/NULL'
            )
        )
    )
    return grupo

In [57]:
cols_asignacion = ['CC', 'CONTRATO', 'NOMBRE_CLIENTE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CARTERA', 'AGENCIA', 'FLAG']
df_asignacion_test = df_asignacion_test[cols_asignacion]

df_asignacion_test['CC'] = df_asignacion_test['CC'].astype('Int64').astype(str).str.zfill(8)
df_asignacion_test['CONTRATO'] = df_asignacion_test['CONTRATO'].astype('Int64').astype(str).str.zfill(18)

df_base_test = df_base_test.merge(df_asignacion_test, on='CC', how='left')
df_base_test.sort_values(by=['CC', 'TIPO_CARTERA'], ascending=[True, False], inplace=True)

filtro = (df_base_test['FLAG'] != 1) & (df_base_test['FLAG'].notna())
df_base_filtro = df_base_test.loc[filtro].groupby('CC').apply(actualizar_tipo_cartera).reset_index(drop=True)

df_base_test = df_base_test.merge(df_base_filtro[['CC', 'TIPO_CARTERA']], on='CC', how='left', suffixes=('', '_y'))
df_base_test['TIPO_CARTERA_FINAL'] = df_base_test['TIPO_CARTERA_y'].fillna(df_base_test['TIPO_CARTERA'])

df_base_test.drop(columns=['TIPO_CARTERA', 'TIPO_CARTERA_y'], inplace=True)
df_base_test.rename(columns={'TIPO_CARTERA_FINAL': 'TIPO_CARTERA'}, inplace=True)
df_base_test.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)

df_base_test['TIPO_CARTERA'] = df_base_test['TIPO_CARTERA'].fillna('NULL')
df_base_test['TIPO_FONDO'] = df_base_test['TIPO_FONDO'].fillna('NULL')

df_base_test['FLAG'] = df_base_test['FLAG'].astype('Int64')
df_base_test['CONTRATO'] = df_base_test['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

cols_base_test = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_base_test = df_base_test[cols_base_test]

print(df_base_test.shape)
print(df_base_test.dtypes)
df_base_test.head(5)

(95, 16)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO         object
IMPORTE           float64
MONEDA             object
NOMBRE             object
FLAG                Int64
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-11-02,01836935,00248,00142,1000.0,PEN,MARCIAL AUGUSTO VASQUEZ SOLIS,1,001101759600256572,NULL,KSTBC,MARCIAL AUGUSTO SOLIS VASQUEZ,05-Nov,MIV,UNSECURED,MORNESE MORA
1,2024-11-04,04295145,00248,00310,11840.0,PEN,PAGO DE PRESTAMO VENCIDO.,<NA>,NaN,NULL,NaN,NaN,05-Nov,MIV,NULL,NaN
2,2024-11-04,13608270,00248,00323,400.0,PEN,STALIM FIDEL FLORES ATENCIO,1,001104869600893425,NULL,BC,STALIN FIDEL FLORES ATENCIO,05-Nov,MIV,SECURED,SIN AGENCIA
3,2024-11-04,14171150,00248,00616,1100.0,PEN,LUIS TARAZONA,<NA>,NaN,NULL,NaN,NaN,05-Nov,MIV,NULL,NaN
4,2024-11-04,14839020,00248,00190,200.0,PEN,OLGA NAVIDAD PEZET OLIVA,1,001101905001003212,NULL,KSTBC,OLGA NATIVIDAD PEZET OLIVA,05-Nov,MIV,UNSECURED,CLASA MORA


In [58]:
if base_count == df_base_test.shape[0]:
    print('REGISTROS OK')
elif base_count > df_base_test.shape[0]:
    print('Se han perdido registros')
else:
    print('Se han duplicado registros: ', df_base_test.shape[0] - base_count)

REGISTROS OK


In [59]:
df_base_no_flag = df_base_test[df_base_test['FLAG'].isnull()]
no_flag_count = df_base_no_flag.shape[0]
print(no_flag_count)
df_base_no_flag.head(5)

29


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
1,2024-11-04,04295145,00248,00310,11840.00,PEN,PAGO DE PRESTAMO VENCIDO.,<NA>,NaN,NULL,NaN,NaN,05-Nov,MIV,NULL,NaN
3,2024-11-04,14171150,00248,00616,1100.00,PEN,LUIS TARAZONA,<NA>,NaN,NULL,NaN,NaN,05-Nov,MIV,NULL,NaN
5,2024-11-04,14982272,00248,00122,2200.00,PEN,00248,<NA>,NaN,NULL,NaN,NaN,05-Nov,MIV,NULL,NaN
9,2024-11-04,21215143,00248,00130,4087.89,PEN,WALTER WILLIAM LA TORRE CHAMBI,<NA>,NaN,NULL,NaN,NaN,05-Nov,MIV,NULL,NaN
12,2024-11-04,21306331,00248,00139,120.00,PEN,BENEDICTO FILEMON PACHECO MAYTA,<NA>,NaN,NULL,NaN,NaN,05-Nov,MIV,NULL,NaN


In [60]:
#################### MONOPRODUCTO ####################

In [61]:
df_mono = df_base_test[df_base_test['FLAG'] == 1]
mono_count = df_mono.shape[0]
print(mono_count)
df_mono.head(5)

53


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-11-02,01836935,00248,00142,1000.0,PEN,MARCIAL AUGUSTO VASQUEZ SOLIS,1,001101759600256572,NULL,KSTBC,MARCIAL AUGUSTO SOLIS VASQUEZ,05-Nov,MIV,UNSECURED,MORNESE MORA
2,2024-11-04,13608270,00248,00323,400.0,PEN,STALIM FIDEL FLORES ATENCIO,1,001104869600893425,NULL,BC,STALIN FIDEL FLORES ATENCIO,05-Nov,MIV,SECURED,SIN AGENCIA
4,2024-11-04,14839020,00248,00190,200.0,PEN,OLGA NAVIDAD PEZET OLIVA,1,001101905001003212,NULL,KSTBC,OLGA NATIVIDAD PEZET OLIVA,05-Nov,MIV,UNSECURED,CLASA MORA
6,2024-11-04,15730269,00248,00266,6566.0,PEN,JOSE OLIMPIO ROJAS BALBUENA.,1,001102379600148946,NULL,BC,JOSE OLIMPIO ROJAS BALBUENA,05-Nov,MIV,SECURED,SIN AGENCIA
7,2024-11-04,20564553,00248,00106,400.0,PEN,SUSANA LAZO ARAGON,1,001101065000703336,NULL,EXJ,SUSANA LAZO ARAGON,05-Nov,MIV,UNSECURED,MORNESE MORA


In [62]:
#################### REACTIVA ####################

In [63]:
df_reactiva = df_mono[(df_mono['TIPO_CARTERA'] == 'UNSECURED') & (df_mono['TIPO_FONDO'].isin(['REACTIVA', 'CRECER', 'FAE']))]

df_reactiva['TIPO_PAGO'] = None
cols_reactiva = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'TIPO_PAGO', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_reactiva = df_reactiva[cols_reactiva]

df_reactiva.sort_values(by=['FECHA', 'CC'], inplace=True)
df_reactiva.reset_index(drop=True, inplace=True)
df_reactiva.to_excel(reactiva, index=False)

reactiva_count = df_reactiva.shape[0]
print(reactiva_count)
df_reactiva.head(5)

3


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,FLAG,CONTRATO,TIPO_FONDO,CARTERA,TIPO_PAGO,FECHA_ENVIO,ID_RESPONSABLE
0,2024-11-04,22729885,00248,00023,28853.2,PEN,VESTICOL SAC,1,001103569600134888,REACTIVA,BC,None,05-Nov,MIV
1,2024-11-04,24173081,00248,00301,2000.0,PEN,CAUDAL INVERSIONES EIRL .,1,001103019601138895,REACTIVA,BC,None,05-Nov,MIV
2,2024-11-04,28936749,00248,00264,5000.0,PEN,EMPRESA COMERCIALIZADORA EXPORTADORA,1,001102759600630869,REACTIVA,BC,None,05-Nov,MIV


In [64]:
#################### FIN REACTIVA ####################

In [65]:
df_no_mono = df_mono[(df_mono['TIPO_CARTERA'] != 'UNSECURED')]
no_mono_count = df_no_mono.shape[0]
print(df_no_mono.shape)
df_no_mono.head(5)

(9, 16)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
2,2024-11-04,13608270,00248,00323,400.00,PEN,STALIM FIDEL FLORES ATENCIO,1,001104869600893425,NULL,BC,STALIN FIDEL FLORES ATENCIO,05-Nov,MIV,SECURED,SIN AGENCIA
6,2024-11-04,15730269,00248,00266,6566.00,PEN,JOSE OLIMPIO ROJAS BALBUENA.,1,001102379600148946,NULL,BC,JOSE OLIMPIO ROJAS BALBUENA,05-Nov,MIV,SECURED,SIN AGENCIA
31,2024-11-04,22217255,00248,00857,2100.55,PEN,"ALONSO NAJARRO TTITO,",1,001101909600300018,NULL,BC,ALONSO NAJARRO TTITO,05-Nov,MIV,SECURED,SIN AGENCIA
67,2024-11-04,23498968,00248,00238,75000.00,PEN,EDGARDO FRANCISCO ZAVALA VALERO /,1,001102389600168004,NULL,BC,EDGARDO FRANCISCO ZAVALA VALERO,05-Nov,MIV,SECURED,SIN AGENCIA
96,2024-11-04,25682500,00248,00140,34338.73,PEN,20536163213 GYAKIVA BUSINESS SAC,1,001101109600133182,NULL,KSTBC,GYAKIVA BUSINESS SAC,05-Nov,MIV,SECURED,SIN AGENCIA


In [66]:
df_mono = df_mono[(df_mono['TIPO_CARTERA'] == 'UNSECURED') & (~df_mono['TIPO_FONDO'].isin(['REACTIVA', 'CRECER', 'FAE']))]

cols_mono = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
df_mono = df_mono[cols_mono]

df_mono.sort_values(by=['FECHA', 'CC'], inplace=True)
df_mono.reset_index(drop=True, inplace=True)
df_mono.to_excel(monoproducto, index=False)

mono_count_final = df_mono.shape[0]
print(df_mono.shape)
df_mono.head(5)

(41, 16)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-11-02,01836935,00248,00142,1000.0,PEN,MARCIAL AUGUSTO VASQUEZ SOLIS,1,001101759600256572,NULL,KSTBC,MARCIAL AUGUSTO SOLIS VASQUEZ,05-Nov,MIV,UNSECURED,MORNESE MORA
1,2024-11-02,21722432,00248,00203,3767.5,PEN,JULIO CESAR ALARCON RAMOS 41793229,1,001108319601379738,NULL,KSTBC,JULIO CESAR ALARCON RAMOS,05-Nov,MIV,UNSECURED,ASESCOM RJ
2,2024-11-02,24263671,00248,00339,1500.0,PEN,DONALD GUTIERREZ ZEA,1,001103399600147529,NULL,BC,DONALD ANTONIO GUTIERREZ ZEA,05-Nov,MIV,UNSECURED,ASESCOM RJ
3,2024-11-02,24518776,00248,00659,100.0,PEN,PRESTAMO PERSONAL..,1,001101319600173685,NULL,BC,ERICA ROSA SAIRE MAMANI,05-Nov,MIV,UNSECURED,ASESCOM RJ
4,2024-11-02,25134384,00248,00339,1000.0,PEN,RAUL DIEGO SIMON,1,001103399600126211,NULL,BC,RAUL LUIS DIEGO SIMON,05-Nov,MIV,UNSECURED,MORNESE MORA


In [67]:
if mono_count == mono_count_final + reactiva_count + no_mono_count:
    print('MONOPRODUCTO OK')
else:
    print('MONOPRODUCTO ERROR')

MONOPRODUCTO OK


In [68]:
#################### MULTIPRODUCTO ####################

In [69]:
df_multi = df_base_test[df_base_test['FLAG'] > 1]
multi_count = df_multi.shape[0]
print(multi_count)
df_multi.head(5)

13


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
14,2024-11-04,21482167,00248,00316,500.0,PEN,SERVICIOS DIESEL GASTULO SRL,2,001103169600134243,REACTIVA,BC,SERVICIOS DIESEL GASTULO SRL,05-Nov,MIV,SECURED,SIN AGENCIA
19,2024-11-04,21514594,00248,00215,5600.0,PEN,"CESAR AUGUSTO RODRIGUEZ PRIETO VALENCIA,",2,001102155003139728,NULL,BC,CESAR AUGUSTO RODRIGUEZ PRIETO VALENCIA,05-Nov,MIV,SECURED,SIN AGENCIA
25,2024-11-02,21572302,00248,00145,1090.0,PEN,CRISTIAN MARCOS CHURA CAPCHA,2,001101455001757351,NULL,KSTBC,CRISTIAN MARCOS CHURA CAPCHA,05-Nov,MIV,UNSECURED,CLASA MORA
33,2024-11-04,22230944,00248,00275,1733.0,PEN,JOSEPH CERNA CHACON,3,001102759600396998,NULL,BC,JOSEPH HARRISS CERNA CHACON,05-Nov,MIV,SECURED,SIN AGENCIA
43,2024-11-04,22657176,00248,00849,500.0,PEN,PRESTAMO CON TERMINALES 6001.,2,001103879600145137,REACTIVA,BC,FERNANDEZ CARTAGENA& ROSILLO ABOGADOS SA,05-Nov,MIV,UNSECURED,ASESCOM RJ


In [70]:
df_no_multi = df_multi[(df_multi['TIPO_CARTERA'] != 'UNSECURED')]
no_multi_count = df_no_multi.shape[0]
print(df_no_multi.shape)
df_no_multi.head(5)

(4, 16)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
14,2024-11-04,21482167,00248,00316,500.0,PEN,SERVICIOS DIESEL GASTULO SRL,2,001103169600134243,REACTIVA,BC,SERVICIOS DIESEL GASTULO SRL,05-Nov,MIV,SECURED,SIN AGENCIA
19,2024-11-04,21514594,00248,00215,5600.0,PEN,"CESAR AUGUSTO RODRIGUEZ PRIETO VALENCIA,",2,001102155003139728,NULL,BC,CESAR AUGUSTO RODRIGUEZ PRIETO VALENCIA,05-Nov,MIV,SECURED,SIN AGENCIA
33,2024-11-04,22230944,00248,00275,1733.0,PEN,JOSEPH CERNA CHACON,3,001102759600396998,NULL,BC,JOSEPH HARRISS CERNA CHACON,05-Nov,MIV,SECURED,SIN AGENCIA
103,2024-11-04,26650043,00248,00142,10000.0,PEN,GBO INGENIEROS S.A.C.,2,001101709600150459,REACTIVA,BC,GBO INGENIEROS SAC,05-Nov,MIV,SECURED,SIN AGENCIA


In [71]:
df_multi = df_multi[df_multi['TIPO_CARTERA'] == 'UNSECURED']

df_multi['CONTRATO'] = None
df_multi['TIPO_FONDO'] = None
df_multi['CARTERA'] = None
df_multi['TIPO_PAGO'] = None

cols_multi = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'TIPO_PAGO', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_multi = df_multi[cols_multi]

df_multi.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], inplace=True)
df_multi.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
df_multi.reset_index(drop=True, inplace=True)
df_multi.to_excel(multiproducto, index=False)

multi_count_final = df_multi.shape[0]
print(df_multi.shape)
df_multi.head(5)

(9, 15)


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,FLAG,CONTRATO,TIPO_FONDO,CARTERA,TIPO_PAGO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE
0,2024-11-02,21572302,00248,00145,1090.0,PEN,CRISTIAN MARCOS CHURA CAPCHA,2,None,None,None,None,CRISTIAN MARCOS CHURA CAPCHA,05-Nov,MIV
1,2024-11-02,27832008,00248,00224,400.0,PEN,MARIA DEL CARMEN LLANCAY QUISPE 02/11/24,2,None,None,None,None,MARIA DEL CARMEN LLANCAY QUISPE,05-Nov,MIV
2,2024-11-04,22657176,00248,00849,500.0,PEN,PRESTAMO CON TERMINALES 6001.,2,None,None,None,None,FERNANDEZ CARTAGENA& ROSILLO ABOGADOS SA,05-Nov,MIV
3,2024-11-04,22657176,00248,00849,500.0,PEN,PRESTAMO CON TERMINALES 5137.,2,None,None,None,None,FERNANDEZ CARTAGENA& ROSILLO ABOGADOS SA,05-Nov,MIV
4,2024-11-04,24651896,00248,00275,521.0,PEN,MIJAEL CIPRIANO.,2,None,None,None,None,MIJAEL CIPRIANO SANTOS ANDRADE,05-Nov,MIV


In [72]:
if multi_count == multi_count_final + no_multi_count:
    print('MULTIPRODUCTO OK')
else:
    print('MULTIPRODUCTO ERROR')

MULTIPRODUCTO OK


In [73]:
#################### NO ENVIADOS ####################

In [74]:
df_no_enviados = pd.concat([df_no_mono, df_no_multi, df_base_no_flag])
df_no_enviados['FLAG'] = df_no_enviados['FLAG'].astype('Int64').fillna(0)

df_no_enviados['CONTRATO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['CONTRATO'], axis=1)
df_no_enviados['TIPO_FONDO'] = df_no_enviados.apply(lambda x: None if x['FLAG'] != 1 else x['TIPO_FONDO'], axis=1)
df_no_enviados['FLAG'] = df_no_enviados['FLAG'].apply(lambda x: 'NE' if x == 0 else x)

df_no_enviados.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
df_no_enviados.reset_index(drop=True, inplace=True)
df_no_enviados.to_excel(no_enviados, index=False)

print(df_no_enviados.shape)
print(df_no_enviados.dtypes)
df_no_enviados.head(5)

(42, 16)
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO         object
IMPORTE           float64
MONEDA             object
NOMBRE             object
FLAG               object
CONTRATO           object
TIPO_FONDO         object
CARTERA            object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,AGENCIA
0,2024-11-02,27335710,00248,00133,1.00,PEN,CORPORACION. DURANSA SAC.,1,001103235000712017,NULL,BC,CORPORACION DURANSASAC,05-Nov,MIV,SECURED,SIN AGENCIA
1,2024-11-02,21524509,00248,00106,5600.00,PEN,DORA ISABEL CARDENAS RAMOS,NE,None,None,NaN,NaN,05-Nov,MIV,NULL,NaN
2,2024-11-02,23616992,00248,00128,5776.86,PEN,PAGO TARJETA,NE,None,None,NaN,NaN,05-Nov,MIV,NULL,NaN
3,2024-11-02,25874888,00248,00109,2175.00,PEN,ALVAREZ VALENCIA VALERA ALEJANDRA,NE,None,None,NaN,NaN,05-Nov,MIV,NULL,NaN
4,2024-11-02,26476829,00248,00306,3000.00,PEN,001103065002979042 REYCOR EIRL,NE,None,None,NaN,NaN,05-Nov,MIV,NULL,NaN


In [75]:
no_eviados = no_mono_count + no_multi_count + no_flag_count
enviados = mono_count_final + multi_count_final + reactiva_count

print('No enviados:', no_eviados)
print('Enviados:', enviados)
print('Total:', no_eviados + enviados)

if base_count == enviados + no_eviados:
    print('REGISTROS OK')
else:
    print('REGISTROS ERROR')

No enviados: 42
Enviados: 53
Total: 95
REGISTROS OK


In [76]:
#################### FORMATO EXCEL ####################

In [77]:
import openpyxl as op
from openpyxl.styles import Font, PatternFill, Alignment

def format_excel(file_path: str, validator: str) -> None:
    workbook = op.load_workbook(file_path)
    sheet = workbook.active
    
    # Definir estilos generales
    general_font = Font(name='Calibri', size=11)
    header_font = Font(name='Calibri', size=11, bold=True, color='FFFFFF')
    alignment_center = Alignment(horizontal='center', vertical='center')
    
    # Definir estilos específicos
    header_font_white = Font(name='Calibri', size=11, bold=True, color='000000')
    header_fill_blue = PatternFill(start_color='002060', end_color='002060', fill_type='solid')
    header_fill_yellow = PatternFill(start_color='FFD965', end_color='FFD965', fill_type='solid')
    header_fill_green = PatternFill(start_color='C4D79B', end_color='C4D79B', fill_type='solid')
    header_fill_orange = PatternFill(start_color='FABF8F', end_color='FABF8F', fill_type='solid')
    
    # Aplicar estilos generales a todas las celdas
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = general_font
    
    # Aplicar estilos al encabezado (fila 1)
    for cell in sheet[1]:
        cell.font = header_font
        cell.alignment = alignment_center
    
    # Aplicar estilos específicos según el validador
    if validator == 'mono' or validator == 'multi' or validator == 'env' or validator == 'noenv':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 12):  # Columnas J-K
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(12, 16):  # Columnas L-O
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'multi_agencias':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 13):  # Columnas J-L
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(13, 16):  # Columnas M-O
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'react_agencias':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 13):  # Columnas J-L
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(13, 15):  # Columnas M-N
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'reactiva':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 13):  # Columnas J-L
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(13, 15):  # Columnas M-N
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(15, 16):  # Columna O
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    workbook.save(file_path)

format_excel(mono_path, "mono")
format_excel(multi_path, "multi_agencias")
format_excel(react_path, "react_agencias")
format_excel(no_enviados_path, "noenv")

In [78]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Abrir archivos?", icon='warning')
if result == 'yes':
    os.startfile(mono_path)
    os.startfile(multi_path)
    os.startfile(react_path)
    os.startfile(no_enviados_path)

root.destroy()

In [79]:
print("monoproducto:", df_mono.shape)
print("multiproducto:", df_multi.shape)
print("reactiva:", df_reactiva.shape)
print("no enviados:", df_no_enviados.shape)

monoproducto: (41, 16)
multiproducto: (9, 15)
reactiva: (3, 14)
no enviados: (42, 16)


In [80]:
hora = datetime.now() + timedelta(hours=1)

minuto_ajustado = 30 if 0 < hora.minute <= 30 else 0
hora = hora.replace(minute=minuto_ajustado, second=0, microsecond=0)

if minuto_ajustado == 0:
    hora += timedelta(hours=1)

periodo = 'PM' if hora.hour >= 12 else 'AM'

hora_formateada = f"{hora.strftime('%H:%M')} {periodo}"
hora_formateada

'12:30 PM'

In [85]:
from multiproducto import CorreoMultiproducto

root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", f"¿Enviar correo MULTIPRODUCTO {fecha}?", icon='warning')
if result == 'yes':
    CorreoMultiproducto(fecha, file_path, hora_formateada).enviar_correo()

root.destroy()

Correo enviado exitosamente.


In [82]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar multiproducto agencias?", icon='warning')
if result == 'yes':
    df_1 = pd.read_excel(f'{file_path}/multi_agencias/RJ.xlsx')
    df_2 = pd.read_excel(f'{file_path}/multi_agencias/CLASA.xlsx')
    df_3 = pd.read_excel(f'{file_path}/multi_agencias/MORNESE.xlsx')
    
    df_1 = df_1[cols_multi]
    df_2 = df_2[cols_multi]
    df_3 = df_3[cols_multi]
    
    df_1['AGENCIA'] = 'ASESCOM RJ'
    df_2['AGENCIA'] = 'CLASA MORA'
    df_3['AGENCIA'] = 'MORNESE MORA'
    
    print("ASESCOM RJ:", df_1.shape)
    print("CLASA MORA:", df_2.shape)
    print("MORNESE MORA:", df_3.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    contratos_rj = df_1['CONTRATO'].notna().value_counts()[True] - 1
    contratos_clasa = df_2['CONTRATO'].notna().value_counts()[True] - 1
    contratos_mornese = df_3['CONTRATO'].notna().value_counts()[True] - 1
    
    print("Contratos RJ:", contratos_rj)
    print("Contratos CLASA:", contratos_clasa)
    print("Contratos MORNESE:", contratos_mornese)
    print("Total:", contratos_rj + contratos_clasa + contratos_mornese)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_agencias = pd.concat([df_1, df_2, df_3])
    df_agencias.dropna(subset=['CONTRATO'], inplace=True)
    df_agencias['CONTRATO'] = df_agencias['CONTRATO'].apply(
        lambda x: str(int(x)).replace(' ', '').zfill(18) 
        if 16 <= len(str(x).replace(' ', '')) <= 18 
        else None)
    df_agencias.dropna(subset=['CONTRATO'], inplace=True)
    df_agencias.reset_index(drop=True, inplace=True)
    
    df_multi = pd.read_excel(multi_path)
    df_multi['AGENCIA'] = 'NULL'
    
    print("Total Multiproducto Original:", df_multi.shape)
    print("Total Multiproducto Agencias:", df_agencias.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_multi_final = pd.concat([df_multi, df_agencias])
    
    df_multi_final['CC'] = df_multi_final['CC'].astype(str).replace(' ', '').astype('Int64').astype(str).str.zfill(8)
    df_multi_final.drop(columns=['TIPO_FONDO', 'CARTERA'], inplace=True)
    df_multi_final.sort_values(by=['CC', 'CONTRATO'], inplace=True)
    df_multi_final.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)
    df_multi_final.reset_index(drop=True, inplace=True)
    
    df_asignacion_test['CC'] = df_asignacion_test['CC'].astype('Int64').astype(str).str.zfill(8)
    df_asignacion_test['CONTRATO'] = df_asignacion_test['CONTRATO'].apply(lambda x: str(int(x)).replace(' ', '').zfill(18) if pd.notna(x) else x)
    
    df_multi_final = df_multi_final.merge(df_asignacion_test, on="CONTRATO", how="left")
    df_multi_final.rename(columns={'CC_x': 'CC'}, inplace=True)
    df_multi_final.drop(columns=['CC_y'], inplace=True)
    
    df_multi_final['CLAVSERV'] = df_multi_final['CLAVSERV'].astype(str).str.zfill(5)
    df_multi_final['CENTROPAGO'] = df_multi_final['CENTROPAGO'].astype(str).str.zfill(5)
    df_multi_final['CONTRATO'] = df_multi_final['CONTRATO'].fillna('NULL')
    df_multi_final['TIPO_FONDO'] = df_multi_final['TIPO_FONDO'].fillna('NULL')
    df_multi_final['CARTERA'] = df_multi_final['CARTERA'].fillna('NULL')
    df_multi_final['TIPO_CARTERA'] = df_multi_final['TIPO_CARTERA'].fillna('NULL')
    df_multi_final['AGENCIA_y'] = df_multi_final['AGENCIA_y'].fillna('NULL')
    
    df_multi_final['AGENCIA_y'] = df_multi_final.apply(
        lambda x: x['AGENCIA_y'] 
        if x['AGENCIA_y'] == x['AGENCIA_x'] else f"{x['AGENCIA_x']}/{x['AGENCIA_y']}", 
        axis=1)
    
    df_multi_final.drop(columns=['AGENCIA_x', 'FLAG_y', 'NOMBRE_CLIENTE_y'], inplace=True)
    df_multi_final.rename(columns={'AGENCIA_y': 'AGENCIA', 'FLAG_x': 'FLAG', 'NOMBRE_CLIENTE_x': 'NOMBRE_CLIENTE'}, inplace=True)
    
    cols_multi_final = ['FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'TIPO_PAGO', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'AGENCIA']
    df_multi_final = df_multi_final[cols_multi_final]
    df_multi_final.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
    df_multi_final.reset_index(drop=True, inplace=True)
    print("Multiproducto final:", df_multi_final.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_no_enviados_multi = df_multi_final[(df_multi_final['CONTRATO'] == 'NULL') | (df_multi_final['TIPO_CARTERA'] != 'UNSECURED')]
    df_no_enviados_multi['AGENCIA'] = df_no_enviados_multi.apply(lambda x: 'AGENCIA NO IDENTIFICA' if x['CONTRATO'] == 'NULL' else x['AGENCIA'], axis=1)
    df_no_enviados_multi.drop(columns=['TIPO_PAGO'], inplace=True)
    print("No enviados Multiproducto:", df_no_enviados_multi.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_reactivas_multi = df_multi_final[(df_multi_final['TIPO_CARTERA'] == 'UNSECURED') & (df_multi_final['TIPO_FONDO'].isin(['REACTIVA', 'CRECER', 'FAE']))]
    df_reactivas_multi.drop(columns=['NOMBRE_CLIENTE', 'TIPO_CARTERA'], inplace=True)
    print("Reactivas Multiproducto:", df_reactivas_multi.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_multi_final = df_multi_final[(df_multi_final['CONTRATO'] != 'NULL') & 
                                    (df_multi_final['TIPO_CARTERA'] == 'UNSECURED') & 
                                    (~df_multi_final['TIPO_FONDO'].isin(['REACTIVA', 'CRECER', 'FAE']))]
    df_multi_final.drop(columns=['TIPO_PAGO'], inplace=True)
    df_multi_final.to_excel(multiproducto, index=False)
    print("Total Multiproducto:", df_multi_final.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_no_enviados_final = pd.concat([df_no_enviados, df_no_enviados_multi])
    
    df_no_enviados_final['CONTRATO'] = df_no_enviados_final['CONTRATO'].fillna('NULL')
    df_no_enviados_final['TIPO_FONDO'] = df_no_enviados_final['TIPO_FONDO'].fillna('NULL')
    df_no_enviados_final['CARTERA'] = df_no_enviados_final['CARTERA'].fillna('NULL')
    df_no_enviados_final['TIPO_CARTERA'] = df_no_enviados_final['TIPO_CARTERA'].fillna('NULL')
    df_no_enviados_final['NOMBRE_CLIENTE'] = df_no_enviados_final['NOMBRE_CLIENTE'].fillna('NULL')
    df_no_enviados_final['AGENCIA'] = df_no_enviados_final['AGENCIA'].fillna('NULL')
    df_no_enviados_final['CLAVSERV'] = df_no_enviados_final['CLAVSERV'].apply(lambda x: str(int(x)).replace(' ', '').zfill(5))
    df_no_enviados_final.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
    df_no_enviados_final.reset_index(drop=True, inplace=True)
    df_no_enviados_final.to_excel(no_enviados, index=False)
    print("Total NO Enviados:", df_no_enviados_final.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    #df_reactiva = pd.read_excel(react_path)
    df_reactiva_final = pd.concat([df_reactiva, df_reactivas_multi])
    df_reactiva_final.to_excel(reactiva, index=False)
    print("Total Reactiva:", df_reactiva_final.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    df_enviados = pd.concat([df_mono, df_multi_final])
    df_enviados['CLAVSERV'] = df_enviados['CLAVSERV'].apply(lambda x: str(int(x)).zfill(5))
    df_enviados['CENTROPAGO'] = df_enviados['CENTROPAGO'].apply(lambda x: str(int(x)).zfill(5))
    df_enviados.sort_values(by=['FECHA', 'FLAG', 'CC'], inplace=True)
    df_enviados.reset_index(drop=True, inplace=True)
    df_enviados.to_excel(si_enviados, index=False)
    print("Total Enviados:", df_enviados.shape)
    
    print("--------------------------------------------------------------------------------------------------------------------------------------------")
    
    format_excel(multi_path, "multi")
    format_excel(react_path, "reactiva")
    format_excel(no_enviados_path, "noenv")
    format_excel(enviados_path, "env")
    
    os.startfile(multi_path)
    os.startfile(react_path)
    os.startfile(no_enviados_path)
    os.startfile(enviados_path)

root.destroy()

In [83]:
df_asignacion_test[df_asignacion_test['CC']=='25655285']

,CC,CONTRATO,NOMBRE_CLIENTE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA,FLAG
105350,25655285,001101575001798266,PANDORA CORP SAC,UNSECURED,NaN,BC,ASESCOM RJ,3
105351,25655285,001101579600154836,PANDORA CORP SAC,UNSECURED,REACTIVA,BC,ASESCOM RJ,3
105352,25655285,001101579600156626,PANDORA CORP SAC,UNSECURED,REACTIVA,BC,ASESCOM RJ,3


In [84]:
df_asignacion_test[df_asignacion_test['CONTRATO']=='001101419600143586']

,CC,CONTRATO,NOMBRE_CLIENTE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA,FLAG
176084,28366126,001101419600143586,FABMEDIC SAC,UNSECURED,REACTIVA,KSTBC,ASESCOM RJ,3
